# Fase #2 - Detección de Malware de Android
* Pablo Alejandro Méndez #19195
* Diana Zaray Corado #191025
* Orlando Osberto Cabrera #19943
* José Javier Hurtarte #19707

In [2]:
# library imports
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier

## Análisis Exploratorio

La idea principal al desarrollar un modelo que permita la detección de malware en Android es poder utilizar un modelo que no solo sea capaz de identificar *malware* mediante los permisos que solicita una aplicación sino también mediante las llamadas a API´s que están realizan. Para ello se estarán utilizando dos conjuntos de datos provistos en *Kaggle* los cuales se describen a continuación.

### Conjunto de datos de Permisos 

In [101]:
data = pd.read_csv('/data/notebook_files/Permission Dataset.csv')
data.head()

,permission.GetuiService.com.glodon.ynjtapp,permission.COLLECT_METRICS,permission.sec.MDM_PHONE_RESTRICTION,permission.MEDIA_MOUNTED,permission.USAGE_ACCESS_SETTINGS,permission.VOIP_BROADCAST_VOIP_INTENTS,permission.GetuiService.com.huamaitel.client.yun,permission.CLIENT,permission.GetuiService.pailiefive.main,permission.RESTART_PxCKAGES,...,permission.GetuiService.me.chunyu.ChunyuDoctor,permission.A4S_SEND,permission.GetuiService.com.melot.game,permission.CONTACT_SETTINGS,permission.ZALO_SERVICE,permission.GetuiService.com.hooray.snm,permission.READ_USAGESTATS,permission.USE_IFAA_MANAGER,permission.READ_MEDIA_STORAGE,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


El primer conjunto de datos a utilizar contiene **11975** observaciones y **1492** *features* que en este caso representan los permisos que solicita una aplicación para ser instalada. A su vez se puede observar que estos datos ya se encuentran vectorizados, es decir, ya existe una representación numéricas de los permisos solicitados por una aplicación, por lo que no será necesario pre-procesarlos. 

In [105]:
data['isMalware'] = data['class'].apply(lambda x: not x)
print(f"{data['isMalware'].value_counts()}")
px.histogram(data, x='isMalware', title='Figura 1. Representación gráfica de la cantidad de tipos de aplicaciones <br />benignas y malware en Permissions Dataset')

False    6000
True     5975
Name: isMalware, dtype: int64


### Conjunto de Llamadas a API's

In [5]:
# Indica si un feature es un permiso o llamada a API
categories = pd.read_csv('/data/workspace_files/Proyecto1/AMDML2.csv')
categories

,transact,API call signature
0,onServiceConnected,API call signature
1,bindService,API call signature
2,attachInterface,API call signature
3,ServiceConnection,API call signature
4,android.os.Binder,API call signature
...,...,...
210,ACCESS_FINE_LOCATION,Manifest Permission
211,SET_WALLPAPER_HINTS,Manifest Permission
212,SET_PREFERRED_APPLICATIONS,Manifest Permission
213,WRITE_SECURE_SETTINGS,Manifest Permission


In [6]:
api_data = pd.read_csv('/data/workspace_files/Proyecto1/AMDML.csv')
# changing the classification value
api_data.head()

/opt/python/envs/default/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning:

Columns (92) have mixed types.Specify dtype option on import or set low_memory=False.



,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,...,READ_CONTACTS,DEVICE_POWER,HARDWARE_TEST,ACCESS_WIFI_STATE,WRITE_EXTERNAL_STORAGE,ACCESS_FINE_LOCATION,SET_WALLPAPER_HINTS,SET_PREFERRED_APPLICATIONS,WRITE_SECURE_SETTINGS,class
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,S
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,S
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,S
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,1,1,0,0,0,S
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,S


In [106]:
api_data['isMalware'] = api_data['class'].apply(lambda x: True if x == 'S' else False)
print(f"{api_data['isMalware'].value_counts()}")
px.histogram(api_data, x='isMalware', title='Figura 1. Representación gráfica de la cantidad de tipos de aplicaciones <br />benignas y malware en API´s Dataset')

False    9476
True     5560
Name: isMalware, dtype: int64


El segundo conjunto de datos a utilizar cuenta con **15036** obsevaciones y **217** *features*. Los *features* corresponden principalmente a llamadas API´s y algunas representan permisos solicitados por una aplicación antes de su instalación. A su vez, al igual que el conjunto de datos anterior se puede observar que estos ya se encuentran vectorizados. 

## Preprocesamiento (codificación, escalamente y normalización, eliminación de features no relevantes)

Para analizar los *features* de los conjuntos de datos se utilizará *Principal Component Analysis* para poder agrupar el conjunto de *features* en una dimensión "manejable", con la que se pueda observar las características relevantes. Para decidir el mejor número de reducción de componentes se utilizará un *scree plot* la cual funciona de manera similar a la gráfica de codo utilizada en **K-Means***

In [8]:
pca = PCA(n_components=20)
pca_data = pca.fit_transform(data.drop(['class', 'isMalware'], axis=1))
pca.explained_variance_

array([2.2405182 , 0.67182515, 0.47083693, 0.30362265, 0.28418862,
 0.24289195, 0.22932909, 0.22001433, 0.20725965, 0.194614 ,
 0.1761142 , 0.16937619, 0.16651285, 0.14357465, 0.13615197,
 0.13302586, 0.12626529, 0.11652723, 0.11345213, 0.10727403])

In [9]:
px.line(pca.explained_variance_)

Como se observa en el gráfico anterior, el número ideal de componentes sería 2, por lo cual se hace *PCA* con 2 para analizar las caracterísitas principales de los datos.

In [10]:
pca = PCA(n_components=2)
pca_data = pca.fit_transform(data.drop(['class', 'isMalware'], axis=1))
pca_features = pd.DataFrame(data=pca_data)
pca_features['isMalware'] = data['isMalware']
pca_features

,0,1,isMalware
0,0.406293,-1.287697,False
1,5.204345,1.525399,False
2,1.247847,-0.613734,False
3,0.816060,-0.788746,False
4,-1.330647,-0.476020,False
...,...,...,...
11970,0.532567,1.079739,True
11971,-1.546048,-0.631908,True
11972,-1.535198,0.579279,True
11973,-1.127666,0.057448,True


In [11]:
px.scatter(pca_features, x=0, y=1, color='isMalware')

En el gráfico anterior se puede observar que se obtienen valores más altos para el caso de las aplicaciones benignas, sin embargo, la mayoría de las aplicaciones en general se concentra en un mismo punto, con lo cual es difícil diferenciar mendiante este análisis aquellas características que sean más predominantes en aplicaciones de tipo *malware* o benignas 

Para realizar PCA para el segundo conjunto de datos es necesario considerar que existen columnas que están en tipo de dato *string* o a su vez contienen valores de tipo null por lo que será necesario cambiar esas observaciones. En este caso se está utilizando -1 para definir aquellos datos de los cuales no se tiene información.

In [12]:
api_data.isnull().sum()

transact 0
onServiceConnected 0
bindService 0
attachInterface 0
ServiceConnection 0
 ..
SET_WALLPAPER_HINTS 0
SET_PREFERRED_APPLICATIONS 0
WRITE_SECURE_SETTINGS 0
class 0
isMalware 0
Length: 217, dtype: int64

In [13]:
api_data['TelephonyManager.getSimCountryIso'] = api_data['TelephonyManager.getSimCountryIso'].replace(to_replace='?', value=-1)
# now have to delete all the nan values
api_data['TelephonyManager.getSimCountryIso'] = api_data['TelephonyManager.getSimCountryIso'].fillna(-1)
api_data['TelephonyManager.getSimCountryIso'] = pd.to_numeric(api_data['TelephonyManager.getSimCountryIso'])

In [14]:
pca = PCA(n_components=18)
pca_data = pca.fit_transform(api_data.drop(['class', 'isMalware'], axis=1))

pca.explained_variance_

array([4.96550004, 1.31701181, 0.82879286, 0.64654354, 0.50890174,
 0.45729664, 0.39492845, 0.37142168, 0.33815098, 0.29578116,
 0.29216798, 0.27083031, 0.25426472, 0.23679176, 0.23443723,
 0.2168579 , 0.21121054, 0.18730077])

In [15]:
px.line(pca.explained_variance_)

Al igual que en la gráfica anterior se observa que el número de componentes óptimos sería 2, por lo que se aplica PCA para 2 componentes para el segundo conjunto de datos.

In [16]:
pca = PCA(n_components=2)
pca_data = pca.fit_transform(api_data.drop(['class', 'isMalware'], axis=1))
pca_features = pd.DataFrame(data=pca_data)
pca_features['isMalware'] = api_data['isMalware']
pca_features

,0,1,isMalware
0,-2.836505,-0.336134,True
1,-2.502625,0.234378,True
2,-2.417511,-0.397331,True
3,0.882529,2.065648,True
4,-0.941421,0.804637,True
...,...,...,...
15031,2.808139,-1.251126,False
15032,0.299657,-0.751118,False
15033,-1.830412,-0.014334,False
15034,2.563917,-0.512241,False


In [17]:
px.scatter(pca_features, x=0, y=1, color='isMalware')

A diferencia de los resultados obtenidos con el conjunto de datos anterior, en este caso sí se puede observar que existe una diferencia entre la agrupación de características para cada uno de los tipos de aplicaciones. Se obseva que las aplicaciones benignas tienden a estar más agrupadas en la parte inferior y con un mayor rango, mientras que las maliciosas se agrupan más en la parte superior izquierda. En este caso debemos de tomar en cuenta que los datos no se encuentran balancenados y puede deberse a esto que se obtenga una mejor definición de las características de cada uno de los tipos de las aplicaciones.  

### Balanceo de data (técnicas de oversampling y undersampling)

In [18]:
benign = api_data[api_data['isMalware'] == False]
benign = benign.sample(n=6000, random_state=42)
rest = api_data[(api_data['isMalware'])]
under_api_sample = pd.concat([benign, rest])

In [19]:
under_api_sample['isMalware'] = under_api_sample['class'].apply(lambda x: True if x == 'S' else False)
print(f"{under_api_sample['isMalware'].value_counts()}")
px.histogram(under_api_sample, x='isMalware')

False    6000
True     5560
Name: isMalware, dtype: int64


Para el balanceo de datos se decidió realizar undersampling únicamente a los datos de api_sample, debido a que para este conjunto de datos se contaba con el doble de datos que no son malware que de los que si son.

En el caso del primer conjunto de datos no es necesario realizar ningún balanceo debido a que estos datos se encuentran casi perfectamente balanceados, por lo que no será necesario utilizar ninguna estrategia de balanceo.

### Selección de las características finales (correlación entre las características)

Para la selección de características, debido a que se está trabajando con datos categórico y el entrenamiento depende completamente de la identificación del uso o no de un permiso o llamada a API, es por ello que para el caso en el que únicamente se tienen 217 *features* se decidió que se estarán utilizando todos los *features* provistos debido a que se busca que los modelos puedan reconocer de una mejor manera las características indistinguibles de cada uno de los tipos de aplicaciones. 

Por su parte, para el otro conjunto de datos (el que contiene información de los permisos) se estará realizando una estrategia de *feature selection* mediante el cálculo de *chi-square* para identificar aquellas características que tienen un mayor impacto en la clasificación de una aplicación de Android.

In [118]:
data = data.drop(columns=["class"], axis=1)

In [128]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

 
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(data.drop(columns=["isMalware"], axis=1),data["isMalware"])

selected_features = pd.DataFrame({
    'feature':data.drop(columns=["isMalware"], axis=1).columns,
    'scores':fs.scores_
})
mean = selected_features['scores'].mean()
std = selected_features['scores'].std()
print('Valor esperado: ',mean)
print('Desviación estándar: ',std)
selected_features.sort_values('scores', ascending=False).iloc[:int(len(selected_features)*0.4)]

Valor esperado:  35.47790747054501
Desviación estándar:  187.6994866084523


,feature,scores
432,permission.MOUNT_UNMOUNT_FILESYSTEMS,3038.256997
1457,permission.CHANGE_WIFI_STATE,2181.367682
314,permission.GET_TASKS,2172.149187
688,permission.SYSTEM_ALERT_WINDOW,1947.165861
21,permission.WRITE_SETTINGS,1873.422367
...,...,...
1129,permission.CONNECTIVITY_INTERNAL,1.812556
1430,permission.PROCESS_INCOMING_CALLS,1.812556
1383,permission.SYSTEM_UI_VISIBILITY_EXTENSION,1.812556
550,permission.CAMERA_ADDON,1.812556


*Chi-square* es una medida que básicamente permite obtener la correlación entre variables categóricas, ya que permite obtener la relación entre las variables independientes(*features*) y la variable dependiente (*target*). Esto permite realizar la selección de features, ya que proporciona aquellas características de las cuales la variable *target* es más dependiente, proporcionando una valor de *chi-square* más alto. 

Por lo tal, para seleccionar los mejores *features* para el conjunto de datos de permisos solicitados por una aplicación se ordenaron cada una de las características con respecto al valor de *chi-square* obtenido y se extrajo el 40% de los *features* con valores más altos, dejando así como resultado un conjunto de características de **596**.

### Separación de datos (55% para entrenamiento, 15% validación y 30% pruebas)

## Implementación de Modelos


Métricas de evaluación para cada modelo (matriz de confusión, accuracy, precisión, recall, F1), se deben explicar que significan sus valores en el contexto del problema de seguridad propuesto.

Evaluación cruzada con K-10 folds para K = 10.

### 1. Inserta tu primer modelo aquí

### 2. Inserta tu segundo modelo aquí

### 3. Inserta tu tercer modelo aquí

### 4. Inserta tu cuarto modelo aquí

## Bibliografía